In [ ]:
# chainerを用いて深層学習を行う

In [ ]:
import sys
import os
import glob
import itertools
import re
import math
import random
import pandas as pd

import numpy as np
import sklearn
print "sklearn_ver",(sklearn.__version__)

from sklearn import linear_model
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler

from chainer import Chain, Variable, cuda, FunctionSet
import chainer.links as L
import chainer.functions as F
from chainer import datasets
from chainer import optimizers
from chainer import iterators
from chainer import training
from chainer.training import extensions
from chainer import report
from chainer import serializers

import chainer
print "chainer_ver",(chainer.__version__)

import json
import pickle
from itertools import chain

In [ ]:
input_dir = ur"dir_name"
output_dir = ur"dir_name"

In [ ]:
# 入力ファイルを読み込む

In [ ]:
input_df = pd.read_csv(input_dir + "flagon_merge.csv", encoding="cp932", low_memory=False)

In [ ]:
# データ型が定義されているファイルを読み込み
type_list = ur"/src/nttd/matsuura/data/type_list.csv"
df_type = pd.read_csv(type_list, encoding="cp932", low_memory=False)

In [ ]:
type_rep = df_type.replace("VARCHAR2", "CHAR")
tmp12 = type_rep.copy()
tmp01 = type_rep.copy()
tmp12.names = map(lambda x: x + "_12", tmp12.names)
tmp01.names = map(lambda x: x + "_01", tmp01.names)

In [ ]:
type_lis_concat = pd.concat([type_rep,tmp12,tmp01]).reset_index(drop =True)

In [ ]:
# 数値項目を抽出
type_num = type_lis_concat.query('types == "NUMBER"')
input_column = set(list(input_df.columns))
num_column = set(list(type_num["names"]))
num_match_lis = list(input_column & num_column)
print "入力ファイルの項目数　：　NUMBER型の項目数　：", len(input_df.columns), "：", len(num_match_lis)
num_column_lis = input_df.loc[:,num_match_lis]

In [ ]:
# CHAR型の項目を抽出
type_char = type_lis_concat.query('types == "CHAR"')
input_column = set(list(input_df.columns))
char_column = set(list(type_char["names"]))
char_match_lis = list(input_column & char_column)
print "入力ファイルの項目数　：　CHAR型の項目数　：", len(input_df.columns), "：", len(char_match_lis)
char_column_lis = input_df.loc[:,char_match_lis]

In [ ]:
# カテゴリデータは連続値と読み取られるため、ダミー変数に変換する
# ダミー変数に変換するためデータ型を"unicode"にする

In [ ]:
df_char_ex_dum = pd.get_dummies(char_column_lis, drop_first=True)

In [ ]:
merge_file = pd.concat([num_column_lis, df_char_ex_dum], axis=1)

In [ ]:
print merge_file.head()

In [ ]:
merge_file_rename = merge_file.rename(columns = {u"目的変数":"zandaka"})
print "目的変数", len(merge_file_rename.query('zandaka == 1'))

In [ ]:
# 説明変数、目的変数への分割
X = merge_file.drop(columns = {u"目的変数"},axis =1)
y_zandaka = merge_file.loc[:,[u"目的変数"]]

In [ ]:
# 数値項目において訓練データの平均値/標準偏差から訓練データ、テストデータのz-scoreを算出し標準化する
# StandardScalerは上記が実現不可のため個別実装
def number_scaler(X_train, X_test):
    
    train_scaler = pd.DataFrame()
    test_scaler = pd.DataFrame()
    
    train_num = X_train.loc[:, wk_num]
    test_num = X_test.loc[:, wk_num]
    
    # 標準化後にカテゴリ項目をマージさせるために定義
    train_char = X_train.loc[:, use_char_column]
    test_char = X_test.loc[:, use_char_column]
    
    # 訓練データの平均値、標準偏差を取得
    for i in train_num.columns:        
        train_mean = train_num[i].mean()
        train_std = train_num[i].std()
        
        # z-scoreを求める
        train_z = pd.DataFrame({i:(train_num[i] - train_mean) / train_std})
        test_z = pd.DataFrame({i:(test_num[i] - train_mean) / train_std})
        
        train_scaler = pd.concat([train_scaler, train_z], axis=1)
        test_scaler = pd.concat([test_scaler, test_z], axis=1)
    
    # カテゴリ項目をマージ
    train_scaler = pd.concat([train_scaler,train_char], axis=1)
    test_scaler = pd.concat([test_scaler,test_char], axis=1)
    
    train_scaler = train_scaler.fillna(0)
    test_scaler = test_scaler.fillna(0)
    
    return train_scaler, test_scaler

In [ ]:
num_column = set(list(type_num["names"]))

X_column = list(X.columns)
wk_num = list(set(X_column) & num_column)
use_char_column = set(X_column) - set(wk_num)

In [ ]:
# 乱数シード
random.seed(0)
# 0:GPU、-1：CPU
device_id = -1

In [ ]:
# 「訓練：テスト：検証　＝　7：2：1」に分割

In [ ]:
# 訓練、テストデータへ分割　（7：3）
(X_train, X_test, y_train_zan, y_test_zan) = train_test_split(X, y_zandaka, test_size=0.3, random_state=0)

In [ ]:
# 検証用データをテストデータから作成　（7：3）
(X_test, X_valid, y_test_zan, y_valid) = train_test_split(X_test, y_test_zan, test_size=0.3, random_state=0)

In [ ]:
# 標準化
X_train_scaler, X_test_scaler = number_scaler(X_train, X_test)

In [ ]:
# テストデータの欠損値を削除した上で目的変数に欠損値が存在しない行を指定する
X_test_scaler_kn = (X_test_scaler.replace([np.inf,-np.inf],np.nan)).dropna()
noinf_ix = list(X_test_scaler_kn.index)
y_test_zan_kn = y_test_zan.loc[noinf_ix,:]

In [ ]:
# chainer読込用に変換
x_train_numpy = np.array(X_train_scaler, dtype = np.float32)
x_test_numpy = np.array(X_test_scaler_kn, dtype = np.float32)
y_train_zan_numpy = np.array(y_train_zan).astype(np.int32)
y_test_zan_numpy = np.array(y_test_zan_kn).astype(np.int32)

In [ ]:
# chainer読込用に一次元配列に整形
y_train_ex = y_train_zan_numpy.reshape(-1)
y_test_ex = y_test_zan_numpy.reshape(-1)

In [ ]:
# 使用デバイスがGPUの場合、cuda変換
if device_id == 0:
    # GPU用にcuda変換
    x_train_gpu = cuda.to_gpu(x_train_numpy)
    x_test_gpu = cuda.to_gpu(x_test_numpy)
    y_train_gpu = cuda.to_gpu(y_train_ex)
    y_test_gpu = cuda.to_gpu(y_test_ex)
    
    print x_train_gpu.dtype
    print x_test_gpu.dtype
    print y_train_gpu.dtype
    print y_test_gpu.dtype
    
    # リスト化
    train_zan = list(zip(x_train_gpu, y_train_gpu))
    test_zan = list(zip(x_test_gpu, y_test_gpu))

# CPUの場合、リスト化のみ行う
else:
    train_zan = list(zip(x_train_numpy, y_train_ex))
    test_zan = list(zip(x_test_numpy, y_test_ex))

In [ ]:
# 検証用データもテストデータと同様に加工

In [ ]:
# 検証用データを標準化
X_train_dum, X_valid_scaler = number_scaler(X_train, X_valid)

In [ ]:
# 検証データの欠損値を削除した上で目的変数に欠損値が存在しない行を指定する
X_valid_scaler_kn = (X_valid_scaler.replace([np.inf,-np.inf],np.nan)).dropna()
noinf_valid = list(X_valid_scaler_kn.index)
y_valid_zan_kn = y_valid.loc[noinf_valid, :]

In [ ]:
# chainer読込用に変換
x_valid_numpy = np.array(X_valid_scaler_kn, dtype = np.float32)
y_valid_zan_numpy = np.array(y_valid_zan_kn).astype(np.int32)

In [ ]:
# 使用デバイスがGPUの場合、cuda変換
if device_id == 0:
    #GPU用にcuda変換
    x_valid_gpu = cuda.to_gpu(x_valid_numpy)
    y_valid_gpu = cuda.to_gpu(y_valid_zan_numpy)

In [ ]:
# 深層学習

In [ ]:
# NNモデル
class MLP(Chain):

    # 層定義
    def __init__(self, n_units, activate, dropout_ratio, lenfeat):
        super(MLP, self).__init__(
            l1 = L.Linear(None, n_units),
            l2 = L.Linear(None, 2),
            
            # 引数は適用する層の入力次元数と合わせる
            bn1 = L.BatchNormalization(lenfeat),
        )
        self.activate = activate
        self.dropout_ratio = dropout_ratio
        self.train = True
    
    # 訓練/検証の切替
    def set_train_state(self, train):
        self.train = train
    
    # 順伝播
    def __call__(self, x):
        
        # 活性化関数　（relu、sigmoid）
        if self.activate == 'relu':
            h1 = F.dropout(F.relu(self.l1(self.bn1(x, test = not self.train))), train=self.train ,ratio = self.dropout_ratio)
        else: 
            h1 = F.dropout(F.sigmoid(self.l1(self.bn1(x, test = not self.train))), train=self.train ,ratio = self.dropout_ratio)
        
        return self.l2(h1)

In [ ]:
class Classifier(L.Classifier):

    def __init__(self, predictor, rowall, lossfun):
        super(Classifier, self).__init__(predictor = predictor, lossfun=lossfun)
        self.tn = self.fn = self.tp = self.fp = 0
        self.call_cnt = 0
        self.row_cnt = rowall

    def __call__(self, x, t):
        super(Classifier, self).__call__(x ,t)
        y = self.predictor(x)
        y_probs = F.softmax(y)        
        predict = y_probs.data.argmax(axis=1)
        #print "predict:ans={}:{}".format(len(predict), len(t.data))
        
        # confusion_matrix
        tn,fn,tp,fp = self.calc_confusion_matrix(predict, t.data, len(t.data))
        # auc
        auc = self.calc_auc(y_probs.data[:,1], t.data)
        report({'auc':auc, 'tn':tn, 'fn':fn, 'tp':tp, 'fp':fp}, self)
        return self.loss

    def calc_confusion_matrix(self, predict, label, datasize):
        """混合行列を計算する
        """
        # 1epoch(データ件数/バッチサイズ)毎に
        # __call__()が呼び出される回数を保持
        self.call_cnt += 1
        if self.row_cnt <= self.call_cnt * datasize:
            self.tn = self.fn = self.tp = self.fp = 0
            self.call_cnt = 0
        for pred, ans in zip(predict, label):
            if pred == ans == 0:
                self.tn+=1
            elif pred != ans == 0:
                self.fn+=1
            elif pred == ans == 1:
                self.tp+=1
            elif pred != ans == 1:
                self.fp+=1
            else:
                print "irregular-pattern! pred:{},ans:{}".format(pred,ans)

        return self.tn, self.fn, self.tp, self.fp

    def calc_auc(self, prob, label):
        """aucを計算する
        """
        n = len(prob)
        a = 0.
        score_prev = float('-inf')
        fp = tp = 0
        fp_prev = tp_prev = 0
        # (prob,label)のセットでprobの値をキーに降順で並替え
        tmp = sorted(zip(prob, label), key=lambda x:x[0], reverse=True)
        tmp_u = zip(*tmp)
        prob_desc = list(tmp_u[0])
        label_desc = list(tmp_u[1])
        for i in range(n):
            if prob_desc[i] != score_prev:
                # TP(FP)の増分がつくる長方形の面積を加算
                a += self.trapezoid(fp, fp_prev, tp, tp_prev)
                score_prev = prob_desc[i]
                fp_prev = fp
                tp_prev = tp

            if label_desc[i] == 1:
                tp += 1
            else:
                fp += 1

        a += self.trapezoid(fp, fp_prev, tp, tp_prev)

        # 最大面積で正規化
        #print "a:{},TP:{},FP:{}".format(a,tp,fp)
        if a == 0. or tp == 0 or fp == 0:
            return 0.
        else:
            return a / (tp * fp)
        
    def trapezoid(self, x1, x2, y1, y2):
        """頂点を結んだ四角形の面積を求める
        """
        width = abs(x1 - x2)
        height = (y1 + y2) / 2.
        return width * height

In [ ]:
# 固定パラメータ

# バッチサイズ
batchsize = 32
# 学習回数
n_epoch = 30

In [ ]:
# 最適化対象のパラメータ
# 他探索候補としてバッチサイズ、隠れ層の数

# layer_range = range(2,6,1)

# dropout率
d_ratio_range = (0.1*x for x in xrange(1, 10))
# 学習率
lr_range = (0.01*x for x in xrange(1, 11))
# WeightDecayの重み
weight_range = (0.0001*x for x in xrange(1, 11))

param_dist = {"n_units":[4,16,32,64,128,256], 
#               "layer_num":layer_range, 
              "activate":['relu', 'sigmoid'],
              "d_ratio":d_ratio_range,
              "optimizer_name":['SGD', 'MomentumSGD', 'Adam'],
              "lr":lr_range,
              "wd":weight_range
             }

In [ ]:
# 直積の組み合わせからパラメータを取得
param_list = list(itertools.product(param_dist['n_units'], param_dist['activate'],param_dist['d_ratio'],
                                    param_dist['optimizer_name'], param_dist['lr'], param_dist['wd']))
print len(param_list)
print type(param_list)

In [ ]:
# 簡易版ランダムサーチ
def search_param(train_data, test_data, x_valid, y_valid):
    
    # 探索結果を詰める
    result = pd.DataFrame()

    # TODO:実行回数は変数に変更
    # ランダムサーチする回数は任意に設定
    for i in xrange(1, 5):
        
        # 実行パラメータ、正解率、損失関数を詰める
        work_lis = []

        # パラメータの組み合わせからランダムに取得
        sel_param = random.choice(param_list)
        # ユニット数、活性化関数、dropout率、アルゴリズム、学習率、WeightDecayの重み　の順番
        print "実行パラメータ", sel_param

        # インスタンス生成
        #clf = L.Classifier(MLP(sel_param[0], sel_param[1], sel_param[2]), lossfun = F.softmax_cross_entropy)
        lenfeat = len(X.columns)
        clf = Classifier(MLP(sel_param[0], sel_param[1], sel_param[2], lenfeat),
                           len(train_data), lossfun=F.softmax_cross_entropy)

        # 使用デバイス定義
        if device_id >= 0:
            chainer.cuda.get_device(device_id).use()
            clf.to_gpu(device_id)
        else:
            clf.to_cpu()

        # 選択されたアルゴリズムでoptimizerを設定
        if sel_param[3] == 'SGD':
            optimizer = optimizers.SGD(lr = sel_param[4])
        elif sel_param[3] == 'MomentumSGD':
            optimizer = optimizers.MomentumSGD(lr = sel_param[4])
        else:
            optimizer = optimizers.Adam()

        optimizer.setup(clf)
        optimizer.add_hook(chainer.optimizer.WeightDecay(sel_param[5]))

        # Iterator
        train_data_iter = iterators.SerialIterator(train_data, batchsize)
        test_data_iter = iterators.SerialIterator(test_data, batchsize, repeat=False, shuffle=False)

        # trainerを用いて学習
        updater = training.StandardUpdater(train_data_iter, optimizer, device = device_id)
        trainer = training.Trainer(updater, (n_epoch, 'epoch'), out = 'result')

        # 評価
        trainer.extend(extensions.Evaluator(test_data_iter, clf, device = device_id))
        # 学習結果を視覚化
        trainer.extend(extensions.LogReport(log_name = "search_param_zan_" + str(i) + ".log", trigger=(1, 'epoch')))
        trainer.extend(extensions.PlotReport(['main/loss', 'validation/main/loss','main/accuracy', 'validation/main/accuracy'], 
                                                 'epoch',  file_name='loss_zan_' + str(i) + '.png'))
        trainer.extend(extensions.PlotReport(['main/auc', 'validation/main/auc'], x_key='epoch', file_name='auc_zan_' + str(i) + '.png'))
        #trainer.extend(extensions.PrintReport(['epoch', 'main/accuracy', 'validation/main/accuracy', 'main/auc', 'validation/main/auc','elapsed_time']), trigger=(1,'epoch'))

        # 学習の実行
        trainer.run()

        # テストデータの正解率/損失関数を取得
        # ※　最上位だけ取得すると飛び値の可能性があるため、上位10位の平均スコアから求める
        valid_data = trainer._extensions['PlotReport'].extension._data
        loss_data = [data for i, data in valid_data['validation/main/loss']]
        best10_loss = sorted(loss_data)[:10]
        loss_avr = float(sum(best10_loss)) / 10

        acc_data = [data for i, data in valid_data['validation/main/accuracy']]
        best10_acc = sorted(acc_data)[:10]
        acc_avr = float(sum(best10_acc)) / 10
        
        #print trainer._extensions.keys()
        valid_data2 = trainer._extensions['PlotReport_1'].extension._data
        auc_data = [data for i, data in valid_data2['validation/main/auc']]
        best10_auc = sorted(auc_data)[:10]
        auc_avr = float(sum(best10_auc)) / 10

        # 実行パラメータ、正解率、損失関数を詰め込む
        work_list = []
        params = list(sel_param)

        for param in params:
            work_list.append(param)
        
        work_list.append(auc_avr)
        work_list.append(loss_avr)
        work_list.append(acc_avr)
        tmp = pd.DataFrame(work_list).T
        result = pd.concat([result, tmp], ignore_index=True)
    
    result.columns = [u"ユニット数", u"活性化関数", u"dropout率", u"アルゴリズム"
                      , u"学習率", u"WeightDecayの重み", "auc", "loss", "accuracy"]
    # ソートはauc：昇順
    result_sort = result.sort_values(by='auc', ascending=False).reset_index(drop =True)
        
    return result_sort

In [ ]:
zan_param = search_param(train_zan, test_zan, x_valid_numpy, y_valid_zan_numpy)

In [ ]:
# パラメータ探索結果を出力
display(zan_param)
output = os.path.join(output_dir, "search_param_zan.csv")
zan_param.to_csv(output, encoding="cp932", index=False)

In [ ]:
# 最適パラメータで再度trainer実行
lenfeat = len(X.columns)
clf = Classifier(MLP(64,'relu', 0.2, lenfeat),
                           len(train_zan), lossfun=F.softmax_cross_entropy)

if device_id >= 0:
    chainer.cuda.get_device(device_id).use()
    clf.to_gpu(device_id)
else:
    clf.to_cpu()

In [ ]:
# optimizer
optimizer = optimizers.SGD(lr=0.07)
optimizer.setup(clf)
optimizer.add_hook(chainer.optimizer.WeightDecay(0.0006))

In [ ]:
# TODO：最適パラメータ取得し再度モデル構築するまでを自動化
# 一旦は動作確認優先のため、ランダムサーチで抽出した最適パラメータを手入力

In [ ]:
# Iterator
train_data_iter = iterators.SerialIterator(train_zan, batchsize)
test_data_iter = iterators.SerialIterator(test_zan, batchsize, repeat=False, shuffle=False)

In [ ]:
# TODO：学習が最大化したら学習を辞め、optimizer出力する実装を追加
#        (現状では過学習モデルが過学習されたまま出力されてしまうため、検証結果が悪くなる)

In [ ]:
# trainerを用いて学習
updater = training.StandardUpdater(train_data_iter, optimizer, device = device_id)
trainer = training.Trainer(updater, (n_epoch, 'epoch'), out = 'result')

# 評価
trainer.extend(extensions.Evaluator(test_data_iter, clf, device = device_id))
# 学習結果を視覚化
trainer.extend(extensions.LogReport(log_name = "best_param_zan.log", trigger=(1, 'epoch')))
trainer.extend(extensions.PlotReport(['main/loss', 'validation/main/loss','main/accuracy', 'validation/main/accuracy'], 
                                                 'epoch',  file_name='loss_zan.png'))

In [ ]:
# 学習の実行
trainer.run()

In [ ]:
# logファイルの読み込み
with open('result/best_param_zan.log') as f:
    logs = json.load(f)

In [ ]:
# 学習経過をプロット　（損失関数）
result_train, result_test = [], []
for log in logs:
    result_train.append(log['main/loss'])
    result_test.append(log['validation/main/loss'])

# グラフ描画
plt.plot(result_train, label='train')
plt.plot(result_test, label='test')
plt.legend()

plt.title("Loss Progress")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

In [ ]:
# 学習経過をプロット　（正解率）
result_train2, result_test2 = [], []
for log in logs:
    result_train2.append(log['main/accuracy'])
    result_test2.append(log['validation/main/accuracy'])

# グラフ描画
plt.plot(result_train2, label='train')
plt.plot(result_test2, label='test')
plt.legend()

plt.title("Accuracy Progress")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
# モデルの保存
#cpu_model = clf.to_cpu()
serializers.save_npz("./model/chainer_zan.model",clf)

In [ ]:
#モデルの読み込み
serializers.load_npz('./model/chainer_zan.model', clf)

In [ ]:
of = open('chainer_zan.pkl', 'wb')
pickle.dump(clf, of)
of.close()

In [ ]:
# モデル検証

In [ ]:
# 検証用モデル
evaluator = clf.copy()
# 検証モードに切替
evaluator.predictor.train = False

In [ ]:
print clf.predictor.train
print evaluator.predictor.train

In [ ]:
# 検証モードで前進計算
y_pre = evaluator.predictor(x_valid_numpy)

In [ ]:
print y_pre.data
print len(y_pre)

In [ ]:
#ソフトマックス算出して分類確率を出力
y_prob = F.softmax(y_pre)
y_prob.data[:,1]

In [ ]:
y_pre2 = y_prob.data.argmax(axis=1)
print y_pre2
print len(y_pre2)
print len(np.where(y_pre2 == 0)[0])
print len(np.where(y_pre2 == 1)[0])

In [ ]:
# aucを出力
precision, recall, threshold = precision_recall_curve(y_valid_zan_kn, y_pre2)
print "precision ",precision
print "recall",recall
print "threshold",threshold

print ""
print classification_report(y_valid_zan_kn, y_pre2)
print ""

print "confusion_matrix:"
print confusion_matrix(y_valid_zan_kn, y_pre2)
print ""

fpr,tpr,thresholds = roc_curve(y_valid_zan_kn, y_prob.data[:,1])
print "auc", round(auc(fpr,tpr),4)

# ROC曲線を出力
plt.plot(fpr, tpr)
plt.title("ROC curve")
plt.xlabel("False Positve Rate")
plt.ylabel("True Positive Rate")
plt.show()